In [2]:
import gfootball.env as football_env
import time
import pprint
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

from FeatureEncoder import *
from ppo import *

import torch.multiprocessing as mp 

def actor(actor_num, center_model, data_queue, signal_queue, arg_dict):
    print("actor {} started".format(actor_num))
    #11_vs_11_easy_stochastic
    #academy_empty_goal_close 300 epi done
    #academy_empty_goal 450 epi done
    model = PPO(arg_dict["lstm_size"])
    model.load_state_dict(center_model.state_dict())
    env = football_env.create_environment(env_name="academy_empty_goal_close", representation="raw", stacked=False, logdir='/tmp/football', write_goal_dumps=False, write_full_episode_dumps=False, render=False)
    fe = FeatureEncoder()
    
    n_epi = 0
    score = 0
    rollout = []
    
    while True:
        env.reset()
        done = False
        steps = 0
#         score = 0
        n_epi += 1
        h_out = (torch.zeros([1, 1, arg_dict["lstm_size"]], dtype=torch.float), 
                 torch.zeros([1, 1, arg_dict["lstm_size"]], dtype=torch.float))
        
        while not done:
            t1 = time.time()
            while signal_queue.qsize() > 0:
                time.sleep(0.02)
            else:
                model.load_state_dict(center_model.state_dict())
                
            obs = env.observation()
            state_dict = fe.encode(obs[0])
            player_state = torch.from_numpy(state_dict["player"]).float().unsqueeze(0).unsqueeze(0)
            ball_state = torch.from_numpy(state_dict["ball"]).float().unsqueeze(0).unsqueeze(0)
            left_team_state = torch.from_numpy(state_dict["left_team"]).float().unsqueeze(0).unsqueeze(0)
            right_team_state = torch.from_numpy(state_dict["right_team"]).float().unsqueeze(0).unsqueeze(0)
            
            h_in = h_out

            state_dict_tensor = {
              "player" : player_state,
              "ball" : ball_state,
              "left_team" : left_team_state,
              "right_team" : right_team_state,
              "hidden" : h_in
            }
            
            with torch.no_grad():
                prob, _, h_out = model(state_dict_tensor)
            m = Categorical(prob)
            a = m.sample().item()

            prev_obs = obs
            obs, rew, done, info = env.step(a)
            additional_r = fe.calc_additional_reward(prev_obs[0], obs[0])
            fin_r = rew*3.0 + additional_r
            
            state_prime_dict = fe.encode(obs[0])
            
            (h1_in, h2_in) = h_in
            (h1_out, h2_out) = h_out
            h_in_np = (h1_in.numpy(), h2_in.numpy())
            h_out_np = (h1_out.numpy(), h2_out.numpy())
            state_dict["hidden"] = h_in_np
            state_prime_dict["hidden"] = h_out_np

            transition = (state_dict, a, fin_r, state_prime_dict, prob[0][0][a].item(), done)
            rollout.append(transition)

            if len(rollout) == arg_dict["rollout_len"]:
                data_queue.put(rollout)
                rollout = []
                
            state_dict = state_prime_dict

            steps += 1
            score += rew

            if done:
                if n_epi % 10 == 0 and actor_num == 0:
                    print("%d, Done, Step %d Reward: %f" % (n_epi, steps, score))
                    score = 0   


def learner(center_model, queue, signal_queue, arg_dict):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = PPO(arg_dict["lstm_size"], device)
    model.load_state_dict(center_model.state_dict())
    model.to(device)
    
    print("learner start")
    
    while True:
        if queue.qsize() > arg_dict["batch_size"]*arg_dict["buffer_size"]:
            signal_queue.put(1)
            data = []
            for j in range(arg_dict["buffer_size"]):
                mini_batch_np = []
                for i in range(arg_dict["batch_size"]):
                    rollout = queue.get()
                    mini_batch_np.append(rollout)
                mini_batch = model.make_batch(mini_batch_np)
                data.append(mini_batch)
            model.train_net(data)
            center_model.load_state_dict(model.state_dict())
            
            if queue.qsize() > arg_dict["batch_size"]*arg_dict["buffer_size"]:
                print(" data remaining. queue size : ", queue.qsize())
            signal_queue.get()
            
        else:
            time.sleep(0.1)
    

if __name__ == '__main__':
    # hyperparameters
    arg_dict = {
        "num_processes": 6,
        "batch_size": 16,   
        "buffer_size": 5,
        "rollout_len": 10,
        "lstm_size" : 196
    }
    
    np.set_printoptions(precision=3)
    np.set_printoptions(suppress=True)
    pp = pprint.PrettyPrinter(indent=4)
    torch.set_num_threads(1)
    
    center_model = PPO(arg_dict["lstm_size"])
    center_model.share_memory()
    data_queue = mp.Queue()
    signal_queue = mp.Queue()
    processes = []
    
    p = mp.Process(target=learner, args=(center_model, data_queue, signal_queue, arg_dict))
    p.start()
    processes.append(p)
    for rank in range(arg_dict["num_processes"]):
        p = mp.Process(target=actor, args=(rank, center_model, data_queue, signal_queue, arg_dict))
        p.start()
        processes.append(p)
        
    for p in processes:
        p.join()
        
        


actor 0 started
actor 1 started
actor 2 started
actor 3 started
actor 4 started
actor 5 started
learner start
 data remaining. queue size :  424
 data remaining. queue size :  344
 data remaining. queue size :  264
 data remaining. queue size :  184
 data remaining. queue size :  104
10, Done, Step 168 Reward: -1.000000


Process Process-8:


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-79290274f905>", line 127, in learner
    time.sleep(0.1)
KeyboardInterrupt


In [3]:
import gfootball.env as football_env
import time
import pprint
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

from FeatureEncoder import *
from ppo import *

import torch.multiprocessing as mp 
import time

model = PPO()
env = football_env.create_environment(env_name="11_vs_11_stochastic", representation="raw", stacked=False, logdir='/tmp/football', write_goal_dumps=False, write_full_episode_dumps=False, render=False)
fe = FeatureEncoder()

n_epi = 0
score = 0
rollout = []

while True:
    env.reset()
    done = False
    steps = 0
#         score = 0
    n_epi += 1
    h_out = (torch.zeros([1, 1, 256], dtype=torch.float), torch.zeros([1, 1, 256], dtype=torch.float))

    while not done:
        t1 = time.time()
        obs = env.observation()
        state_dict = fe.encode(obs[0])
        player_state = torch.from_numpy(state_dict["player"]).float().unsqueeze(0).unsqueeze(0)
        ball_state = torch.from_numpy(state_dict["ball"]).float().unsqueeze(0).unsqueeze(0)
        left_team_state = torch.from_numpy(state_dict["left_team"]).float().unsqueeze(0).unsqueeze(0)
        right_team_state = torch.from_numpy(state_dict["right_team"]).float().unsqueeze(0).unsqueeze(0)

        h_in = h_out

        state_dict_tensor = {
          "player" : player_state,
          "ball" : ball_state,
          "left_team" : left_team_state,
          "right_team" : right_team_state,
          "hidden" : h_in
        }


        with torch.no_grad():
            prob, _, h_out = model(state_dict_tensor)
        m = Categorical(prob)
        a = m.sample().item()

        prev_obs = obs
        obs, rew, done, info = env.step(a)
#         additional_r = calc_additional_reward(prev_obs[0], obs[0])
#         rew = rew*5.0 + additional_r

        state_prime_dict = fe.encode(obs[0])

        (h1_in, h2_in) = h_in

        steps += 1
        score += rew

        if done:
            if n_epi % 10 == 0 and actor_num == 0:
                print("%d, Done, Step %d Reward: %f" % (n_epi, steps, score))
                score = 0   

{'steps_left': 500, 'right_team_tired_factor': array([0.17098057, 0.73962474, 0.71469581, 0.7004298 , 0.63528061,
       0.74949718, 0.73441845, 0.80784899, 0.7151792 , 0.74715447,
       0.66241133]), 'right_team_active': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]), 'ball_direction': array([ 0.        , -0.        ,  0.00755861]), 'left_team_yellow_card': array([False, False, False, False, False, False, False, False,  True,
       False, False]), 'left_team': array([[-0.99080884, -0.        ],
       [-0.49406919, -0.16490568],
       [-0.56433153, -0.00449594],
       [-0.5791589 ,  0.11604264],
       [-0.69860792,  0.25398785],
       [-0.23589799, -0.07949962],
       [-0.37677947,  0.04122039],
       [-0.26360479,  0.12881984],
       [-0.0438931 , -0.22039051],
       [ 0.39742985,  0.0207978 ],
       [-0.07744762,  0.22872889]]), 'ball_rotation': array([ 0., -0.,  0.]), 'ball_owned_player': -1, 'right_team_yellow_card': array([F

{'steps_left': 484, 'right_team_tired_factor': array([0.17129815, 0.74442768, 0.71855724, 0.70498979, 0.63938451,
       0.75255811, 0.73840237, 0.81176823, 0.71751666, 0.75073886,
       0.66473651]), 'right_team_active': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]), 'ball_direction': array([ 0.03143736, -0.00134959, -0.20086348]), 'left_team_yellow_card': array([False, False, False, False, False, False, False, False,  True,
       False, False]), 'left_team': array([[-8.65431368e-01,  8.28656703e-05],
       [-4.23646897e-01, -1.53542742e-01],
       [-4.92732495e-01,  1.60718970e-02],
       [-5.74982226e-01,  1.57424644e-01],
       [-6.07703090e-01,  2.77913749e-01],
       [-2.44957283e-01, -4.35166880e-02],
       [-3.75420451e-01,  8.08319151e-02],
       [-2.65469223e-01,  1.79127067e-01],
       [-9.33291167e-02, -1.94471464e-01],
       [ 2.94633538e-01,  4.55376953e-02],
       [ 1.23465434e-02,  2.24694088e-01]]), 'ball_rotati

{'steps_left': 470, 'right_team_tired_factor': array([0.1717872 , 0.75009269, 0.72527838, 0.71233308, 0.64602757,
       0.75816602, 0.74575841, 0.81671023, 0.72105151, 0.75477219,
       0.66819787]), 'right_team_active': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]), 'ball_direction': array([-0.02602635, -0.00058151,  0.07049864]), 'left_team_yellow_card': array([False, False, False, False, False, False, False, False,  True,
       False, False]), 'left_team': array([[-8.32074046e-01,  1.00044003e-02],
       [-3.31421971e-01, -1.51434511e-01],
       [-3.99084806e-01,  1.02395574e-02],
       [-4.84373599e-01,  1.72315091e-01],
       [-5.09890854e-01,  2.86696076e-01],
       [-1.86765581e-01, -2.38554198e-02],
       [-3.59232455e-01,  9.83204544e-02],
       [-1.67951390e-01,  2.09778115e-01],
       [ 6.86152431e-04, -1.90272599e-01],
       [ 1.68539479e-01,  1.08320542e-01],
       [ 1.30955845e-01,  2.22390369e-01]]), 'ball_rotati

{'steps_left': 455, 'right_team_tired_factor': array([0.17663306, 0.75475621, 0.72898555, 0.71763796, 0.65009654,
       0.76184809, 0.75076449, 0.82031322, 0.72641158, 0.75862449,
       0.67182839]), 'right_team_active': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]), 'ball_direction': array([ 0.03931095, -0.00251069,  0.07533097]), 'left_team_yellow_card': array([False, False, False, False, False, False, False, False,  True,
       False, False]), 'left_team': array([[-0.83629459,  0.02122828],
       [-0.20573185, -0.15081586],
       [-0.28776836,  0.00615936],
       [-0.36572498,  0.17617159],
       [-0.37875214,  0.2899119 ],
       [-0.14830069,  0.00340738],
       [-0.28092468,  0.10030898],
       [-0.08777624,  0.22489974],
       [ 0.06304752, -0.20323299],
       [ 0.18928321,  0.10603866],
       [ 0.09783085,  0.25693256]]), 'ball_rotation': array([-1.63232271e-05,  4.62032549e-05, -6.95788185e-04]), 'ball_owned_player': -1

{'steps_left': 440, 'right_team_tired_factor': array([0.18252236, 0.76036149, 0.73263419, 0.72502553, 0.65795541,
       0.76428592, 0.75730836, 0.82414424, 0.73135841, 0.76198268,
       0.67455149]), 'right_team_active': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]), 'ball_direction': array([-0.03254156, -0.02232367,  0.51693726]), 'left_team_yellow_card': array([False, False, False, False, False, False, False, False,  True,
       False, False]), 'left_team': array([[-0.82790607,  0.00167893],
       [-0.10998145, -0.15256013],
       [-0.19232441, -0.00147408],
       [-0.26995346,  0.16865487],
       [-0.27789232,  0.28757879],
       [-0.07611077, -0.00942326],
       [-0.20673989,  0.09566677],
       [-0.13887171,  0.22891159],
       [ 0.15046655, -0.18328094],
       [ 0.31557578,  0.09618082],
       [ 0.19277455,  0.26442748]]), 'ball_rotation': array([ 4.90033599e-05, -3.80100464e-05, -6.44803280e-03]), 'ball_owned_player': -1

...

{'steps_left': 238, 'right_team_tired_factor': array([0.21149474, 0.80446357, 0.78539604, 0.78447974, 0.73358381,
       0.81315345, 0.8223967 , 0.88221776, 0.77794284, 0.81145334,
       0.72533178]), 'right_team_active': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]), 'ball_direction': array([-0.0240216 ,  0.00181902, -0.06544714]), 'left_team_yellow_card': array([False, False, False, False, False, False, False, False,  True,
       False, False]), 'left_team': array([[-0.85924172,  0.01315347],
       [-0.06179664, -0.17437358],
       [-0.03175367, -0.01672635],
       [-0.24406166,  0.1144223 ],
       [-0.22432578,  0.25889847],
       [ 0.10263047, -0.03645412],
       [-0.09735139,  0.05802376],
       [ 0.24287157,  0.12790665],
       [ 0.13013983, -0.22561669],
       [ 0.26584956,  0.05771534],
       [ 0.22387204,  0.31135651]]), 'ball_rotation': array([-4.23603669e-05, -5.33447273e-05, -7.87763484e-03]), 'ball_owned_player': -1

{'steps_left': 225, 'right_team_tired_factor': array([0.21543938, 0.80630147, 0.78798246, 0.78726232, 0.73707694,
       0.81471819, 0.82627827, 0.88685036, 0.78238273, 0.81646359,
       0.72817677]), 'right_team_active': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]), 'ball_direction': array([-0.00540677,  0.00128845,  0.04659954]), 'left_team_yellow_card': array([False, False, False, False, False, False, False, False,  True,
       False, False]), 'left_team': array([[-0.8968178 ,  0.01219294],
       [-0.00465499, -0.14269592],
       [ 0.12581985, -0.01265752],
       [-0.31390715,  0.13131562],
       [-0.06325756,  0.26757106],
       [ 0.16000646,  0.0550036 ],
       [-0.07842254,  0.11915987],
       [ 0.10387183,  0.19111268],
       [ 0.20164895, -0.19526966],
       [ 0.33752188,  0.10332814],
       [ 0.23424636,  0.29954517]]), 'ball_rotation': array([ 1.55819762e-05,  2.83245136e-05, -1.83751195e-04]), 'ball_owned_player': 5,

{'steps_left': 211, 'right_team_tired_factor': array([0.21561027, 0.80826581, 0.79267991, 0.78962642, 0.73989689,
       0.81694055, 0.82943237, 0.89214462, 0.78421688, 0.81854737,
       0.72912574]), 'right_team_active': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]), 'ball_direction': array([ 0.00102475,  0.00116146, -0.26929149]), 'left_team_yellow_card': array([False, False, False, False, False, False, False, False,  True,
       False, False]), 'left_team': array([[-0.98416936,  0.00421423],
       [-0.13167228, -0.13464591],
       [ 0.09179036, -0.0161816 ],
       [-0.26025641,  0.13821167],
       [-0.14274026,  0.24969171],
       [ 0.09181683,  0.07925749],
       [-0.14632517,  0.1348619 ],
       [-0.0060205 ,  0.16639552],
       [ 0.1870372 , -0.18450983],
       [ 0.27694538,  0.0840685 ],
       [ 0.16817011,  0.3127704 ]]), 'ball_rotation': array([-7.14188718e-06,  1.10551837e-05, -4.42161036e-05]), 'ball_owned_player': 9,

{'steps_left': 197, 'right_team_tired_factor': array([0.21561027, 0.81218565, 0.79784834, 0.79314166, 0.74414372,
       0.82069474, 0.83284724, 0.89631665, 0.78807628, 0.82017756,
       0.72960293]), 'right_team_active': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]), 'ball_direction': array([-0.00429256, -0.01547855, -0.22933964]), 'left_team_yellow_card': array([False, False, False, False, False, False, False, False,  True,
       False, False]), 'left_team': array([[-0.96078646, -0.00781196],
       [-0.24678344, -0.10887069],
       [-0.09819587, -0.02429301],
       [-0.21997033,  0.10488353],
       [-0.2068813 ,  0.24134147],
       [ 0.00221316,  0.02654324],
       [-0.12085557,  0.09660498],
       [-0.06119521,  0.14766325],
       [ 0.05993088, -0.18512079],
       [ 0.20678252,  0.06096724],
       [ 0.13095105,  0.3076548 ]]), 'ball_rotation': array([ 8.29674827e-05, -1.18577473e-05, -5.65272407e-04]), 'ball_owned_player': -1

KeyboardInterrupt: 

In [6]:
import numpy as np

x = np.array([False, False, False, False, False, False, False, False,  True,
       False, True])

print(np.sum(x))

2
